<a href="https://colab.research.google.com/github/ccnmagnoo/colabs/blob/main/sec_etl_professional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Resume
Actually there´s 11.700 profesional certificated in GAS and Electricity procedures. But Just 290 of them are women, and [SEC](www.sec.cl) (institution focused in regulate this profesionals), dosen´t focus in this labor. This code is used for extract and filter those women to use in diverse social programs, and visualice their labor and effort in this energy field

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
SOURCE_WEB = 'https://wlhttp.sec.cl/buscadorinstaladores/buscador.do'
PATH_GDRIVE = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality'
eli_source = f'{PATH_GDRIVE}/sec_eli.xls'
gas_source = f'{PATH_GDRIVE}/sec_gas.xls'

#Preparing Dataset

In [ ]:
#read gas & dataframe it
gas_df = pd.read_excel(gas_source,header=4)
gas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              1898 non-null   object
 1   CORREO ELECTRÓNICO  1867 non-null   object
 2   RUT                 1898 non-null   object
 3   DIRECCIÓN           1898 non-null   object
 4   TELÉFONO            902 non-null    object
 5   CELULAR             1844 non-null   object
 6   LICENCIA CLASE      1898 non-null   object
dtypes: object(7)
memory usage: 103.9+ KB


In [ ]:
#read electrica & dataframe it
eli_df = pd.read_excel(eli_source,header=4)
eli_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9881 entries, 0 to 9880
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              9881 non-null   object
 1   CORREO ELECTRÓNICO  9611 non-null   object
 2   RUT                 9881 non-null   object
 3   DIRECCIÓN           9881 non-null   object
 4   TELÉFONO            4465 non-null   object
 5   CELULAR             9614 non-null   object
 6   LICENCIA CLASE      9881 non-null   object
dtypes: object(7)
memory usage: 540.5+ KB


In [ ]:
#joint gas + eli
all_technicians = pd.concat([eli_df,gas_df])
all_technicians.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11779 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              11779 non-null  object
 1   CORREO ELECTRÓNICO  11478 non-null  object
 2   RUT                 11779 non-null  object
 3   DIRECCIÓN           11779 non-null  object
 4   TELÉFONO            5367 non-null   object
 5   CELULAR             11458 non-null  object
 6   LICENCIA CLASE      11779 non-null  object
dtypes: object(7)
memory usage: 736.2+ KB


#Definig Transform Functions

In [ ]:
from pandas.io.formats.style_render import non_reducing_slice
from pandas.core.arrays import boolean
#name normalizer
from typing import Union, List
from enum import Enum
import numpy as np

#IMPORTANT : constant used in keras model version 2
CHARS:List[str] = list('aábcdeéfghiíjklmnñoópqrstuúvwxyz')
NAME_BLOCK_SIZE = 20

class Gender(Enum):
  MALE = 'male'
  FEMALE='female'

#normalize name for keras neuronal net consumption
def nameSerializer(unitName:str,size:int=20)->Union[List[float],None]:
  if unitName == None:
    return None

  name_array = list(unitName)
  # [f,i,r,s,t] ->[6/23,8/23,9/23,11/23,20/23] indexation
  descriteName = list(map(lambda char:CHARS.index(char)/len(CHARS),name_array))
  # content into a 20-size box [0.0 ,0.0 ,....0.0]
  box = [0.0]*(size-len(descriteName))
  input = np.concatenate((np.array(box),np.array(descriteName)))
  return list(input.flatten())

#extract just the first name without divisions
class Composer(Enum):
  FIRST = 0
  SECOND = 1

def nameNormalizer(input:str|None,compose:Composer=Composer.FIRST)->Union[str,None]:
  #single word name
  if input == None:
    return None
  #single name checker
  #if len(input.split(' '))==1 and compose == Composer.SECOND:
    #compose = Composer.FIRST

  decomposed_name:str = input.strip().split(' ')[compose.value if len(input.split(' '))>1 else 0].lower()
  #remove al not alpha
  return ''.join([ char for char in decomposed_name if char.isalpha()])

#import ML keras tensorflow model
import keras
from keras.optimizers import get
MODEL = 'keras_genderizer_model_v2.h5'
MODEL_PATH = f'/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/{MODEL}'
model = keras.models.load_model(MODEL_PATH)

#wrapping gender
def getGenderByKeras(unitName:str|None)->Union[Gender,None]:
  if unitName == None:
    return None

  options=[Gender.MALE,Gender.FEMALE]
  input = nameSerializer(unitName,20)

  prediction = model.predict([input])[0]
  category = options[np.argmax(prediction)]
  return category

#for saving process simplified selector with just "o" or "a"
def getGenderSimplified(unitName:str|None)->Gender | None:
  if not unitName:
    return None

  last_char = unitName[-1]
  if last_char == 'o':
    return Gender.MALE
  if last_char == 'a':
    return Gender.FEMALE
  return None

def check_wrong_positive(unitName:str|None)-> bool:
  WRONG_POSITIVE = 'iván,victor,alexis,matthew,williams,alan,jonnhatan,félix,néstor,alex,robinson,valentín,osmar,víctor,dragomir,jehu,rony,raymer,joshua,rienzi,thomas,frankz,steve,werner,tomás,yamil,felix,nicola,bladimir,steve,andree,oragel,jeffrie,'
  w_list:list[str] = str.split(WRONG_POSITIVE,',')
  return unitName in w_list

#main general function wrapper
def getGender(name:str | None)->Gender | None:
  #checking input is empty
  if name ==  None:
    return None

  #try simplified calculus, for speed 🚀
  firstname_check = getGenderSimplified(nameNormalizer(name))
  if firstname_check:
    return firstname_check.value

  #second try with second name
  second = getGenderSimplified(nameNormalizer(name,Composer.SECOND))
  if second:
    return second.value

  #catching wrong positive
  if check_wrong_positive(nameNormalizer(name)):
    return Gender.MALE

  #if simplified dosen´t work applys de keras model
  modeler  = getGenderByKeras(nameNormalizer(name))
  if modeler:
    return modeler.value

  return None

In [ ]:
len('Carlos'.split(' '))
getGender('Victr')

1/1 [==============================] - 0s 28ms/step


'male'

#Transform Dataset & calculating columns

In [ ]:
# Transform
## split NOMBRE column
all_technicians[['Apellidos','Nombres']] = all_technicians['NOMBRE'].str.split(',',1,expand=True)
## remove null names
all_technicians['Nombres'] = all_technicians['Nombres'].map(lambda name: name.strip())
all_technicians

<ipython-input-40-939b2ce165c1>:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  all_technicians[['Apellidos','Nombres']] = all_technicians['NOMBRE'].str.split(',',1,expand=True)


,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 de Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés
3,"Calderón Barraza , Nabor Alejandro",ncalderon@cjenergy.cl,13977385-3,"avenida rica aventura 11544, Antofagasta",055950899,099591296,A,Calderón Barraza,Nabor Alejandro
4,"Araya Reyes , Jecar Exequiel",jecar.araya@gmail.com,17478701-8,"Pasaje 1 62, Nueva Aurora, Viña del Mar, Vina ...",NaN,956841025,B,Araya Reyes,Jecar Exequiel
...,...,...,...,...,...,...,...,...,...
1893,"Hernández Araya , Alexis Duvan",alexishernandez.araya@gmail.com,19612379-2,"Avenida los puelches 921, Chillan",NaN,978706209,1,Hernández Araya,Alexis Duvan
1894,"Rivas Navarrete , Miguel Eduardo",alambreart@gmail.com,17747817-2,"mulchen 383, instalador de redes de gas ,clase...",413351174,930893107,3,Rivas Navarrete,Miguel Eduardo
1895,"Quinillao Carvallo , Henrri Aníbal",henrriq@gmail.com,13734890-K,"malvoa 939, Temuco",NaN,97155737,3,Quinillao Carvallo,Henrri Aníbal
1896,"Sunkel Ojeda , Juan Carlos",jcsunkel@gmail.com,7641852-7,"Pasaje Navidad 114, Villa Cataluña, Los Ángeles",NaN,978521408,1,Sunkel Ojeda,Juan Carlos


##Detecting Genger

In [ ]:
all_technicians['gender'] = all_technicians['Nombres'].map(getGender)

In [ ]:
from pandas import DataFrame
result:DataFrame = all_technicians.loc[all_technicians['gender']=='female']
result

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres,gender
111,"Godoy Tello , Nicolle Angélica",nicolle.godoyt@gmail.com,18239928-0,"Huamachuco 10040, Antofagasta",552211946,962191811,A,Godoy Tello,Nicolle Angélica,female
150,"Lazo Núñez , Cristina Andrea",acmiproyelectric@gmail.com,16118331-8,"Lo encalda 106, Nunoa",NaN,56930397940,B,Lazo Núñez,Cristina Andrea,female
186,"Moraga González , Nicola Andrés",moraga.gonzalez.n@gmail.com,19418946-K,"Anibal pinto 9697, El Bosque",NaN,86113959,A,Moraga González,Nicola Andrés,female
193,"Villarroel Anwandter , Natalia Elisa",natito.a@gmail.com,16106156-5,"malloco 247, Valparaíso",NaN,986309701,D,Villarroel Anwandter,Natalia Elisa,female
231,"Herrera Contreras , María Mercedes",Herrerasu.23@gmail.com,18757199-5,"Av septimo de linea 3396, H, Antofagasta",NaN,944450364,A,Herrera Contreras,María Mercedes,female
...,...,...,...,...,...,...,...,...,...,...
1830,"Rojas Araya , Violeta Del Rosario",violetarojasaraya177@gmail.com,15095378-2,"Simón alamos 94, 123, Torre 1, Quillota",NaN,930549211,3,Rojas Araya,Violeta Del Rosario,female
1852,"Esparza Mardones , Yasna Evelin",yasnaesparzamardones@gmail.com,15167666-9,"Conferencia 1083, casa 3048, Santiago",NaN,73153933,3,Esparza Mardones,Yasna Evelin,female
1859,"Pedraza Quezada , Patricia Andrea",ppedrazaq@gmail.com,15838098-6,"Camino de Los Torques 50, Condominio la Fuente...",229466959,971092044,1,Pedraza Quezada,Patricia Andrea,female
1864,"Garcés Fuentealba , Yasmín Antonia",YASMIN_GARCES@HOTMAIL.COM,16735192-1,"LOS PUQUIOS KM 11, CAMINO LOS PUQUIOS KILOMETR...",82643012,982643012,3,Garcés Fuentealba,Yasmín Antonia,female


In [ ]:
#transform functions

#extract city
def extract_last_col(dir:str)->str|None:
  dir_list = list(reversed(dir.split(',')))
  if len(dir_list)==0:
    return None
  return dir_list[0]


#name case format
import re

def namecase(string:str)->str:
  s_list = re.split('\s+',string.strip())
  for index, word in enumerate(s_list):
    s_list[index] = word[0].upper() + word[1:].lower()
  return ' '.join(s_list)

def setType(input_type:str)->str:
  if input_type == '1 y 4':
    return 'GAS'
  if input_type.isnumeric():
    return 'GAS'
  else:
    return 'ELI'


#Get Territory
from difflib import SequenceMatcher
DPA_PATH = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/DPA2018.xlsx'
DPA = pd.read_excel(DPA_PATH)
from pandas import Series

class Territory(Enum):
  CITY='comuna'
  PROVINCE='prox'
  REGION='region'

def getTerritory (query:str,req:Territory=Territory.CITY,res:Territory=Territory.REGION)->Series:
  #generate a serie with ratio similatities value
  ratio_series:DataFrame = DPA[req.value].apply(lambda req: SequenceMatcher(None,query,req).ratio())
  #find the most simliar ratio an get this index
  most_similar_index = ratio_series.idxmax()
  #extract row index
  result:Series = DPA.iloc[most_similar_index]
  #return value require, like city, region or province
  return result[res.value]


In [ ]:
getTerritory('Quillomta',res=Territory.REGION)

'De Valparaíso'

In [ ]:
result['Comuna'] = result['DIRECCIÓN'].map(extract_last_col)
result['Comuna'] = result['Comuna'].apply(lambda c:getTerritory(c,res=Territory.CITY))
result['Región'] = result['Comuna'].apply(lambda c:getTerritory(c,res=Territory.REGION))
result['CORREO ELECTRÓNICO'] = result['CORREO ELECTRÓNICO'].str.lower()

result['Categoría'] = result['LICENCIA CLASE'].map(setType)
for field in ['Nombres','Apellidos','DIRECCIÓN','Comuna']:
  result[field] = result[field].map(namecase)

result

<ipython-input-164-eee2063d4bbc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Comuna'] = result['DIRECCIÓN'].map(extract_last_col)
<ipython-input-164-eee2063d4bbc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Comuna'] = result['Comuna'].apply(lambda c:getTerritory(c,res=Territory.CITY))
<ipython-input-164-eee2063d4bbc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres,gender,Comuna,Categoría,Región
111,"Godoy Tello , Nicolle Angélica",nicolle.godoyt@gmail.com,18239928-0,"Huamachuco 10040, Antofagasta",552211946,962191811,A,Godoy Tello,Nicolle Angélica,female,Antofagasta,ELI,De Antofagasta
150,"Lazo Núñez , Cristina Andrea",acmiproyelectric@gmail.com,16118331-8,"Lo Encalda 106, Nunoa",NaN,56930397940,B,Lazo Núñez,Cristina Andrea,female,Requínoa,ELI,Del Libertador B. O'Higgins
186,"Moraga González , Nicola Andrés",moraga.gonzalez.n@gmail.com,19418946-K,"Anibal Pinto 9697, El Bosque",NaN,86113959,A,Moraga González,Nicola Andrés,female,El Bosque,ELI,Metropolitana de Santiago
193,"Villarroel Anwandter , Natalia Elisa",natito.a@gmail.com,16106156-5,"Malloco 247, Valparaíso",NaN,986309701,D,Villarroel Anwandter,Natalia Elisa,female,Valparaíso,ELI,De Valparaíso
231,"Herrera Contreras , María Mercedes",herrerasu.23@gmail.com,18757199-5,"Av Septimo De Linea 3396, H, Antofagasta",NaN,944450364,A,Herrera Contreras,María Mercedes,female,Antofagasta,ELI,De Antofagasta
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,"Rojas Araya , Violeta Del Rosario",violetarojasaraya177@gmail.com,15095378-2,"Simón Alamos 94, 123, Torre 1, Quillota",NaN,930549211,3,Rojas Araya,Violeta Del Rosario,female,Quillota,GAS,De Valparaíso
1852,"Esparza Mardones , Yasna Evelin",yasnaesparzamardones@gmail.com,15167666-9,"Conferencia 1083, Casa 3048, Santiago",NaN,73153933,3,Esparza Mardones,Yasna Evelin,female,Santiago,GAS,Metropolitana de Santiago
1859,"Pedraza Quezada , Patricia Andrea",ppedrazaq@gmail.com,15838098-6,"Camino De Los Torques 50, Condominio La Fuente...",229466959,971092044,1,Pedraza Quezada,Patricia Andrea,female,Colina,GAS,Metropolitana de Santiago
1864,"Garcés Fuentealba , Yasmín Antonia",yasmin_garces@hotmail.com,16735192-1,"Los Puquios Km 11, Camino Los Puquios Kilometr...",82643012,982643012,3,Garcés Fuentealba,Yasmín Antonia,female,El Carmen,GAS,De Ñuble


In [ ]:
result_path = f'{PATH_GDRIVE}/sec_result.xlsx'
result.to_excel(result_path)

In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291 entries, 111 to 1865
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              291 non-null    object
 1   CORREO ELECTRÓNICO  278 non-null    object
 2   RUT                 291 non-null    object
 3   DIRECCIÓN           291 non-null    object
 4   TELÉFONO            122 non-null    object
 5   CELULAR             280 non-null    object
 6   LICENCIA CLASE      291 non-null    object
 7   Apellidos           291 non-null    object
 8   Nombres             291 non-null    object
 9   gender              291 non-null    object
 10  Comuna              291 non-null    object
 11  Categoría           291 non-null    object
 12  Región              291 non-null    object
dtypes: object(13)
memory usage: 31.8+ KB


In [ ]:
result_path_v = f'{PATH_GDRIVE}/sec_result_v.xlsx'
result.loc[result['Región']=='De Valparaíso'].to_excel(result_path_v)